### 3D Skeleton Visualization

In [12]:
import os
import json
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact
import copy
from scipy.spatial.transform import Rotation as R
from test_utils import *

from test_utils import get_rootrel_pose, _weak_project, camera_to_image_frame, _infer_box, optimize_scaling_factor

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.gridspec as gridspec

plt.switch_backend('TkAgg')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42


### Load outputs

In [3]:
motionbert_root = "/home/hrai/codes/MotionBERT"
mb_output_root = motionbert_root + "/output"
mb_aihub_output_root = mb_output_root + "/aihub"
assert os.path.exists(mb_aihub_output_root) == True # check if the path exists

In [5]:
model1_path = mb_aihub_output_root + "/MB_ft_h36m_30_M160A_3"
model2_path = mb_aihub_output_root + "/FT_MB_ft_h36m_MB_ft_aihub_sport_243_dazzling-shape-17_30_M160A_3"
model1_path, model2_path

('/home/hrai/codes/MotionBERT/output/aihub/MB_ft_h36m_30_M160A_3',
 '/home/hrai/codes/MotionBERT/output/aihub/FT_MB_ft_h36m_MB_ft_aihub_sport_243_dazzling-shape-17_30_M160A_3')

In [25]:
frame_num = 100

In [26]:
# load model1 output
mp4, npy = os.listdir(model1_path)
assert 'mp4' in mp4 # check if mp4 file exists
assert 'npy' in npy # check if npy file exists
output = np.load(os.path.join(model1_path, npy))
model1_pred = output[frame_num]
model1_pred_hat = get_rootrel_pose(model1_pred)

# load model2 output
mp4, npy = os.listdir(model2_path)
assert 'mp4' in mp4 # check if mp4 file exists
assert 'npy' in npy # check if npy file exists
output = np.load(os.path.join(model2_path, npy))
model2_pred = output[frame_num]
model2_pred_hat = get_rootrel_pose(model2_pred)

In [27]:
fig = plt.figure(0, figsize=(10, 10))
ax = plt.axes(projection="3d")
ax.set_xlim(-512, 512)
ax.set_ylim(-512, 512)
ax.set_zlim(-512, 512)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=12., azim=80)
#show3Dpose(model1_pred_hat*1000, ax)
show3Dpose(model2_pred_hat*1000, ax)
plt.show()

### Load GT

In [10]:
from zipfile import ZipFile

In [11]:
aihub_root = "/home/hrai/Datasets/HAAI/AIHUB"
aihub_3d_gt_path = os.path.join(aihub_root, "label/train/[라벨]3D_json.zip")
aihub_3d_gt_path

'/home/hrai/Datasets/HAAI/AIHUB/label/train/[라벨]3D_json.zip'

In [12]:
# load zip file
zip_label = ZipFile(os.path.join(aihub_3d_gt_path), 'r')
list_input = zip_label.namelist()

In [13]:
action = '30'
actor = 'M160A'
frame = '30'
# actor = 'M180D' #'M160A'
# frame = '311' # '30'
load_path = "3D_json/{}_{}/3D_{}_{}_{}.json".format(action, actor, action, actor, frame)
gt = json.loads(zip_label.read(load_path).decode('utf-8'))

In [14]:
print(gt.keys())
print(gt['info'].keys())
print(gt['annotations'].keys())

dict_keys(['info', 'annotations'])
dict_keys(['supercategory', 'action_category_id', 'actor_id', '3d_pos', '3d_rot'])
dict_keys(['frame_no', 'obj_path', '3d_pos', '3d_rot', 'trans_params'])


### Load camera parameter

In [15]:
aihub_camera_param_path = os.path.join(aihub_root, "label/train/Camera_json_train.zip")

In [16]:
# load zip file
zip_param = ZipFile(os.path.join(aihub_camera_param_path), 'r')
list_input = zip_param.namelist()

In [17]:
action = '30'
# #actor = 'M180D' 
# cam_num = '4'
actor = 'M160A'
cam_num = '3'
load_path = "Camera_json/train/{}_{}_{}.json".format(action, actor, cam_num)
camera_param = json.loads(zip_param.read(load_path).decode('utf-8'))

In [18]:
camera_param

{'camera_date': '20201008',
 'camera_no': 3,
 'extrinsics': [[-0.733395875, 0.026952436, 0.679267645, -8.36987305],
  [-0.155711249, -0.979308605, -0.129261598, 740.389099],
  [0.66172874, -0.200569466, 0.722417653, 4432.7334]],
 'intrinsics': [[0.68166077, 0.0, 0.50988585],
  [0.0, 0.68166077, 0.26416245],
  [0.0, 0.0, 1.0]]}

In [19]:
W = 1920
H = 1080
extrinsic_properties = np.array(camera_param['extrinsics'])
R = copy.deepcopy(np.array(camera_param['extrinsics'])[:,:3])
T = copy.deepcopy(np.array(camera_param['extrinsics'])[:,3])
R_c = R.T
C = - np.matmul(R_c, T)
intrinsic_properties = np.array(camera_param['intrinsics']) # normalized intrinsic matrix
intrinsic_properties[:2, :] *= W # denormalize
fx = intrinsic_properties[0,0]
fy = intrinsic_properties[1,1]
cx = intrinsic_properties[0,2]
cy = intrinsic_properties[1,2]

### Visualize GT

In [20]:
# aihub to h36m pose
world_3d = aihub2h36m(np.array(gt['annotations']['3d_pos'])[:, :3].reshape(1, 24, 3))[0]
world_3d.shape

(17, 3)

In [21]:
fig = plt.figure(0, figsize=(10, 10))
ax = plt.axes(projection="3d")
min_, max_ = -2000, 2000
ax.set_xlim(min_, max_)
ax.set_ylim(min_, max_)
ax.set_zlim(min_, max_)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=12., azim=80)
show3Dpose(world_3d, ax)
plt.show()

### Calculate scaling factor

In [22]:
# world to camera
pos = copy.deepcopy(world_3d)
cam_3d = World2CameraCoordinate(pos, extrinsic_properties) # World coordinate -> Camera coordinate
cam_3d_hat = get_rootrel_pose(cam_3d)

In [23]:
fig = plt.figure(0, figsize=(10, 10))
ax = plt.axes(projection="3d")
min_, max_ = -2000, 2000
ax.set_xlim(min_, max_)
ax.set_ylim(min_, max_)
ax.set_zlim(min_, max_)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=12., azim=80)
show3Dpose(cam_3d_hat, ax)
plt.show()

In [24]:
# camera to image
box = _infer_box(cam_3d, {'fx': fx, 'fy': fy, 'cx': cx, 'cy': cy}, 0)
img_2d, img_3d = camera_to_image_frame(cam_3d, box, {'fx': fx, 'fy': fy, 'cx': cx, 'cy': cy}, 0) 
img_2d_hat = get_rootrel_pose(img_2d) # (17, 2) # root-relative pose 
img_3d_hat = get_rootrel_pose(img_3d) # (17, 3) # root-relative pose 

In [25]:
# https://github.com/Vegetebird/MHFormer/blob/main/demo/vis.py
def get_2d_pose_image(kps, img, box=None):
    connections = [[0, 1], [1, 2], [2, 3], [0, 4], [4, 5],
                   [5, 6], [0, 7], [7, 8], [8, 9], [9, 10],
                   [8, 11], [11, 12], [12, 13], [8, 14], [14, 15], [15, 16]]

    LR = np.array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], dtype=bool)

    lcolor = (255, 0, 0)
    rcolor = (0, 0, 255)
    thickness = 3

    for j,c in enumerate(connections):
        start = map(int, kps[c[0]])
        end = map(int, kps[c[1]])
        start = list(start)
        end = list(end)
        cv2.line(img, (start[0], start[1]), (end[0], end[1]), lcolor if LR[j] else rcolor, thickness)
        cv2.circle(img, (start[0], start[1]), thickness=-1, color=(0, 255, 0), radius=3)
        cv2.circle(img, (end[0], end[1]), thickness=-1, color=(0, 255, 0), radius=3)
        if box is not None:
            box = box.astype(int)
            cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

    return img

def plot_cv2_image(img):
    #plt.axis('off')
    plt.imshow(img)
    plt.show()

In [26]:
import cv2

In [27]:
# visualize 2D pose
img_path = "/home/hrai/Datasets/HAAI/AIHUB/30_M160A_3/30_M160A_3_30.jpg"
#img_path = "/home/hrai/Datasets/HAAI/AIHUB/30_M180D_4/30_M180D_4_311.jpg"
#img = np.ones([1080, 1920, 3])
img = cv2.imread(img_path)
img = get_2d_pose_image(img_2d, img, box)
plot_cv2_image(img)

In [28]:
# img_2d
img = np.ones([2000, 2000, 3])
img = get_2d_pose_image(cam_3d_hat[:, :2] + 1000, img)
plot_cv2_image(img)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [29]:
# optimize scaling factor
pred_lambda = optimize_scaling_factor(img_3d_hat, cam_3d_hat) # x,y,z 사용
pred_lambda

3.3579845428466797

In [30]:
img_25d = img_3d * pred_lambda
img_25d_hat = get_rootrel_pose(img_25d)

In [31]:
fig = plt.figure(0, figsize=(10, 10))
ax = plt.axes(projection="3d")
ax.set_xlim(-512, 512)
ax.set_ylim(-512, 512)
ax.set_zlim(-512, 512)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=12., azim=80)

show3Dpose(img_25d_hat, ax)
show3Dpose(cam_3d_hat, ax)

plt.show()

### Visualize inference and gt

In [44]:
pred_hat_scaled = pred_hat * pred_lambda

In [45]:
fig = plt.figure(0, figsize=(10, 10))
ax = plt.axes(projection="3d")
ax.set_xlim(-512, 512)
ax.set_ylim(-512, 512)
ax.set_zlim(-512, 512)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=12., azim=80)

show3Dpose(img_25d_hat, ax)
show3Dpose(pred_hat_scaled, ax)

plt.show()

### Calculate MPJPE for single pose

In [42]:
import torch
from test_utils import MPJPE_for_single_pose, MPJPE_for_multiple_pose

In [43]:
mpjpe = MPJPE_for_single_pose(img_25d_hat, cam_3d_hat)
print(mpjpe, "mm")

13.47767607231927 mm


In [46]:
mpjpe = MPJPE_for_single_pose(img_25d_hat, pred_hat_scaled)
print(mpjpe, "mm")

105.43126661780063 mm


## For multiple frames

In [36]:
# whitelist
action_list = ['30']
actor_list = ['M160A']
frame_list = ['30']

#list_input = zip_label.namelist()
for action in action_list:
    for actor in actor_list:
        for frame in frame_list:
            print(action, actor, cam_num)
            load_path = "3D_json/{}_{}/3D_{}_{}_{}.json".format(action, actor, action, actor, frame)
            print(load_path)
            data_label = json.loads(zip_label.read(load_path).decode('utf-8'))

30 M160A 3
3D_json/30_M160A/3D_30_M160A_30.json
